# Time Resolution Scan Analysis

This notebook use the template created with [template_scan_analysis.ipynb](template_scan_analysis.ipynb). follow initial instructions there and run the notebook till the end.

Let us quickly check if the file is really here:

In [ ]:
!ls pulse_template.npz
!ls /mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*.fits.fz

Load the needed libraries.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from digicampipe.io.event_stream import calibration_event_stream
import numpy as np
from glob import glob
from tqdm import tqdm #  progress bar
from ctapipe.instrument import CameraGeometry
from ctapipe.visualization import CameraDisplay
from astropy import units as u
from scipy.stats.distributions import chi2

We read the pulse template data and compute the boolean 2D array which indicates which indexes from the template to match camera data for all possibles offset between 0 an 4 ns (1 line per offset, 1 column per sample).
We plot the pulse for the different offsets.

As we see large offsets cut the template pulse, we pad the template with 0 so it does not happen.

In [ ]:
template_pulse = np.load('pulse_template.npz')
x = template_pulse['x']
delta_x = x[1]-x[0]
y = template_pulse['y']
dy = template_pulse['dy']
print('loaded', template_pulse.keys(), 'from pulse template.')
print('template with', len(x), 'points from t={:.4f} ns'.format(x[0]),
      'to t={:.4f} ns'.format(x[-1]), 'delta x={:.4f} ns'.format(delta_x))
oversampling = int(np.round(4 / delta_x)) #  samples in data are separated by 4ns
print('template was oversampled ', oversampling, 'times compared to data.')

n_sample = 50
max_shift = 4 
n_offset = int(max_shift * oversampling)
offsets = np.arange(n_offset) * delta_x
for idx in range(n_offset):
    offsets_idx = np.arange(idx, len(y), oversampling)
    plt.plot(y[offsets_idx], '-')
    plt.xlabel('sample')
    plt.ylabel('normalized amplitude')
    plt.title('samples for ' + str(n_offset) + ' offsets ({:.1f} ns)'.format(n_offset * delta_x))

plt.figure()
x_padded = np.hstack([np.zeros(max_shift*oversampling), x])
y_padded = np.hstack([np.zeros(max_shift*oversampling), y])
dy_padded = np.hstack([np.ones(max_shift*oversampling)*np.inf, dy])
boolean_template_offset = np.zeros([n_offset, len(y_padded)], dtype=bool)
for idx in range(n_offset):
    offsets_idx = np.arange(idx, len(y_padded), oversampling)
    boolean_template_offset[idx, offsets_idx] = True
    plt.plot(y_padded[offsets_idx], '-')
    plt.xlabel('sample')
    plt.ylabel('normalized amplitude')
    plt.title('samples for ' + str(n_offset) + ' offsets ({:.1f} ns)'.format(n_offset * delta_x))

We read the raw camera files.
For each event we normalize the pulse and roughly align it with the template.

We then compute the $\chi^2$ between the measured pulse and the template pulse. We average over all events and store an individual $\chi^2$ value for each AC LEDs'DAC level, each time offset and each pixel.

We store also the mean and the standart deviaton of the integrated pulse over all events for each LED level and each pixel.


In [ ]:
AC_levels = np.arange(0, 750, 5)
n_level = len(AC_levels)
input_files = sorted(glob(
    '/mnt/baobab/sst1m/raw/2018/05/25/SST1M_01/SST1M_01_20180525_*.fits.fz'
))
print(len(input_files), 'files for', n_level, 'AC levels')
assert len(input_files) == n_level, "ERROR: the number of AC LED levels and the number of files do not match."

electronic_noise = 0.8 # we take 0.8 LSB as electronic noise
zero_pe_integral = 10
gain_integral = 22

n_pixel = 1296
Rough_factor_between_single_pe_amplitude_and_integral = 21 / 5.8
chi_squared = np.zeros([n_level, n_pixel, n_offset])
mean_integral_for_level = np.zeros([n_level, n_pixel])
std_integral_for_level = np.zeros([n_level, n_pixel])
all_integral_events = []
print()
for level_idx, level in enumerate(AC_levels):
    print('analyzing file', input_files[level_idx], 'for level', level, 'LSB')
    events = calibration_event_stream([input_files[level_idx]])#, max_events=300)
    n_events = np.zeros(n_pixel, dtype=np.int)
    integral_events = []
    for e in events:
        adcs = e.data.adc_samples[:, 10:30] - e.data.digicam_baseline[:, None]
        integrals = adcs.sum(axis=1)
        null_charge = (integrals - zero_pe_integral) / gain_integral < 2.5  #  at least 0.8 pe
        n_events += ~null_charge
        integral_events.append(integrals)
        noise_normalized = np.zeros([n_pixel,])
        non_null_adcs_norm = adcs[~null_charge, :] / integrals[~null_charge, None] * Rough_factor_between_single_pe_amplitude_and_integral
        non_null_noise_normalized = electronic_noise / integrals[~null_charge] * Rough_factor_between_single_pe_amplitude_and_integral
        for idx in range(n_offset):
            offsets_idx = np.arange(idx, len(y_padded), oversampling)
            template = y_padded[offsets_idx]
            squared_template_error = (dy_padded[None, offsets_idx])**2 + (noise_normalized[~null_charge, None])**2
            squared_diff_data_template = (non_null_adcs_norm[:, :len(template)] - template[None, :])**2
            assert np.any(np.isnan(squared_template_error)) == False
            assert np.any(np.isnan(squared_diff_data_template)) == False
            assert np.all(squared_template_error > 0)
            chi_squared[level_idx, ~null_charge, idx] += np.sum(squared_diff_data_template/squared_template_error, axis=1)/len(template)
            #print((integrals[~null_charge] - zero_pe_integral) / gain_integral)
            #print(np.sum(~null_charge))
            #plt.plot(adcs_normalized.transpose())
            #plt.plot(template, 'k-')
            #break
        #break
    #break
    chi_squared[level_idx, n_events>0, :] /= n_events[n_events>0, None]
    chi_squared[level_idx, n_events==0, :] = np.nan
    integral_events = np.array(integral_events)
    mean_integral_for_level[level_idx, :] = np.mean(integral_events, axis=0)
    std_integral_for_level[level_idx, :] = np.std(integral_events, axis=0)
    all_integral_events.append(integral_events)
    
all_integral_events = np.array(all_integral_events)
pe = (mean_integral_for_level - zero_pe_integral) / gain_integral
d_pe = (std_integral_for_level - zero_pe_integral) / gain_integral
np.savez('chi2.npz', all_integral_events=all_integral_events, pe=pe, d_pe=d_pe, chi_squared=chi_squared)
print('done')
pass

We plot the measured charge fluctuations function of the measured charge. We see the fluctuation is proportional to $\sqrt{p.e.}$ as expected from a Poissonian process until ~700 p.e.

We also plot for each pixel the relation between the LEDs AC level and the measured charge. The limit before saturation (700 p.e.) is shown in red.

In [ ]:
data = np.load(chi2.npz)
all_integral_events=data['all_integral_events']
pe=data['pe']
d_pe=data['d_pe']
chi_squared=data['chi_squared']
del data
"""
events_pe = (all_integral_events - zero_pe_integral) / gain_integral
plt.figure(figsize=(12, 6))
plt.hist(events_pe.flatten(), np.arange(-10.5/22, 10, 1/22))
plt.xlabel('charge [p.e.]')
plt.ylabel('counts')
plt.yscale('log', nonposy='clip')
"""

plt.figure(figsize=(12, 6))
#plt.plot(pe.flatten(), d_pe.flatten() ,'+', label='measured')
plt.scatter(pe.flatten(), d_pe.flatten(), 1, np.log10(np.nanmin(chi_squared, axis=2)).flatten(), label='measured')
plt.plot(np.logspace(-1, 3, 100), np.sqrt(np.logspace(-1, 3, 100)), 'k--', label='$\sqrt{p.e.}$')
plt.plot([700, 700], [0, np.max(d_pe)], 'r--')
plt.xlabel('mean charge measured [p.e.]')
plt.ylabel('std charge measured [p.e.]')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
cbar = plt.colorbar()
cbar.set_label('$\log_{10}(\chi^2)$')
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(AC_levels, pe)
plt.plot([AC_levels[0], AC_levels[-1]], [700, 700], 'r--')
plt.xlabel('DAC level of AC LEDs')
plt.ylabel('charge measured [p.e.]')
pass

We look for bad pixels by looking pixels where the minimimum $\chi^2$ is larger than 10.

In [ ]:
print(chi_squared.shape)
min_chi_squared_per_pixel = np.nanmin(np.nanmin(chi_squared, axis=0), axis=-1)
problematic_pixels = min_chi_squared_per_pixel > 10)
best_level_idx = np.argmin(np.min(chi_squared, axis=2), axis=0).astype(np.int)
#problematic_pixels = np.logical_or(problematic_pixels, AC_levels[best_level_idx] <= 100)

print(np.sum(problematic_pixels), 'problematic pixels:', np.arange(n_pixel)[problematic_pixels])
plt.figure(figsize=(12, 6))
plt.semilogy(np.arange(n_pixel)[~problematic_pixels], 
             min_chi_squared_per_pixel[~problematic_pixels], '+', label='good pixel')
plt.semilogy(np.arange(n_pixel)[problematic_pixels],
             min_chi_squared_per_pixel[problematic_pixels], 'rx', label='bad pixel')
plt.xlabel('pixel #')
plt.ylabel('minimum $\chi^2$')
plt.legend()

plt.figure(figsize=(12, 6))
#plt.plot(pe[:, ~problematic_pixels].flatten(), d_pe[:, ~problematic_pixels].flatten() ,'+', label='good pixel')
plt.scatter(pe[:, ~problematic_pixels].flatten(), d_pe[:, ~problematic_pixels].flatten(), 5, 
            np.log10(np.nanmin(chi_squared, axis=2))[:, ~problematic_pixels].flatten(), marker='+', label='measured')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
plt.xlim(0.1, 1000)
plt.ylim(0.1, 50)
#plt.clim(0, 100)
cbar = plt.colorbar()
cbar.set_label('$\log_{10}(\chi^2)$')
plt.plot(pe[:, problematic_pixels].flatten(), d_pe[:, problematic_pixels].flatten() ,'rx', label='bad pixel') 
plt.plot(np.arange(0, 1000, 0.1),np.sqrt(np.arange(0, 1000, 0.1)), 'k--', label='$\sqrt{p.e.}$')
plt.xlabel('mean p.e. measured')
plt.ylabel('std p.e. measured')
plt.legend()
pass

To check if the light level of the CTS was fine, we look for each LED which AC level gave the minimum $\chi^2$ and check there was some margin. We see some pixels have still their best $\chi^2$ for the highest light level, probably because of faulty LEDs. 

In [ ]:
best_level_idx = np.nanargmin(np.nanmin(chi_squared[:, ~problematic_pixels, :], axis=2), axis=0).astype(np.int)
plt.figure(figsize=(12, 6))
plt.plot(np.arange(n_pixel)[~problematic_pixels], AC_levels[best_level_idx], '+')
plt.xlabel('pixel #')
plt.ylabel('AC level [LSB]')
plt.ylim([min(AC_levels)-10, max(AC_levels)+10])
pass

Now lets look at how the $\chi^2$ evolves with the time offset. We look for the best light level for each pixel. They are nice parabolas. The offset giving the minimum $\chi^2$ is the result of the fit. The $1\sigma$ uncertancy on the offset corresponds to the variation of offset needed to increase $\chi^2$ by 1. 

We can get aproximate best offset of each pixel just by taking the offset with the minimum $\chi^2$. 

In [ ]:
chi_squared_max = 10
pixels_ok = np.arange(n_pixel)[~problematic_pixels]
n_pixel_ok = len(pixels_ok)
print(n_pixel_ok, 'good pixels. ploting chi2 for the best level for each of them.')
chi_squared_best_offset = np.nanmin(chi_squared[:, ~problematic_pixels, :], axis=2)
best_levels_idx = np.nanargmin(chi_squared_best_offset, axis=0).astype(np.int)
chi_squared_best_level = np.zeros([n_pixel_ok, n_offset])
for pixel_ok, level_idx in enumerate(best_levels_idx):
    chi_squared_best_level[pixel_ok, :] = chi_squared[level_idx, pixels_ok[pixel_ok], :]
    

#print(chi_squared_best_level[2, :])
#print(chi_squared[best_level_idx[2], 2, :])
pixel_in_fig = np.any(chi_squared_best_level<chi_squared_max, axis=1)
level_in_fig = np.any(chi_squared_best_level<chi_squared_max, axis=0)
offsets_in_fig = offsets[level_in_fig]

plt.figure(figsize=(12, 6))
plt.plot(offsets, chi_squared_best_level.transpose(),'-')
plt.xlabel('offset [ns]')
plt.ylabel('$\chi^2 /(N-1)$')
plt.xlim([np.min(offsets_in_fig)-.1, np.max(offsets_in_fig)+.1])
plt.ylim([0, chi_squared_max])

# plot the best offset as the minimum of chi2, depricated in favor or the estimation based on a parabola fit 
"""
index_best_offsets = np.nanargmin(chi_squared_best_level, axis=-1)
print('ploting the best offset for', len(index_best_offsets), 'good pixels')
best_offsets_per_pixel = offsets[index_best_offsets]
plt.figure(figsize=(12, 6))
plt.plot(pixels_ok, best_offsets_per_pixel, '+')
plt.xlabel('pixel #')
plt.ylabel('best offset [ns]')
"""
pass

To get a more precise result and error estimation we fit a parabola around the minimum $\chi^2$ for the best offsets.
We plot an histogram of the residuals.

In [ ]:
points_rel = np.arange(-6, 7, 1)
n_point_fit = len(points_rel)
# we create a boolean mask keeping only the points around the min chi2 for each level and each pixel
index_best_offsets = np.argmin(chi_squared_best_level, axis=1)
index_best_offsets[index_best_offsets < -np.min(points_rel)] = -np.min(points_rel)
index_best_offsets[index_best_offsets >= n_offset - np.max(points_rel)] =  n_offset - np.max(points_rel) -1
fit_points_bool = np.zeros_like(chi_squared_best_level, dtype=bool)
for k in points_rel:
    fit_points_bool = np.logical_or(fit_points_bool, np.eye(n_offset,k=k)[index_best_offsets])

# we fit parabolas
chi2_fitted = chi_squared_best_level[fit_points_bool].reshape(n_pixel_ok, n_point_fit)
polys, residuals, rank, singular_values, rcond = np.polyfit(points_rel*delta_x, chi2_fitted.transpose(), 2, full=True)

plt.figure(figsize=(12, 6))
h, xh, _ = plt.hist(residuals, np.linspace(0, 5, 100))
plt.xlabel('residuals')
plt.ylabel('counts')
pass

As there are a few fits which failed, we only keep fits with residuals below 5.
We calculate the offset and the error form the parameters of the parabola fit.

In [ ]:
fit_ok = residuals < 5
print('pixels with problematic fit:', pixels_ok[~fit_ok])
offset_fit_rel = np.ones(n_pixel_ok) * np.nan
# offset_fit is the abscice of the minimum of the parabola
offset_fit_rel[fit_ok] = -0.5 * polys[1, fit_ok] / polys[0, fit_ok]
offset_fit = offset_fit_rel + best_offsets_per_pixel

plt.figure(figsize=(12, 6))
plt.plot(pixels_ok, offset_fit, '+')
plt.xlabel('pixel #')
plt.ylabel('offset [ns]')

# d_offset_fit is how much we need to change the offset to increase chi2 by 1 (for 1 sigma confidence level)
d_offset_fit = np.ones(n_pixel_ok) * np.nan
d_offset_fit[fit_ok] = np.sqrt(1./polys[0, fit_ok]) 
chi_squared_fit_min = polys[2, :]

plt.figure(figsize=(12, 6))
plt.plot(np.polyval(polys, offset_fit_rel - d_offset_fit) - chi_squared_fit_min, '+',
                    label='$\Delta \chi^2 (\Delta t = \sigma_t)$')
plt.plot([0, 1295], [1, 1], 'k--')
plt.title('$\Delta \chi^2$ at $1 \sigma$ (should be ~1)')
plt.xlabel('pixels #')
plt.ylabel('$\Delta \chi^2$')
plt.legend()

plt.figure(figsize=(12, 6))
plt.plot(pixels_ok, d_offset_fit, '+')
plt.xlabel('pixel #')
plt.ylabel('uncertaincy on offset [ns]')
pass

Lets try to understand the patern of the measured offset.

No obvious correlation with sectors nor boards nor pixel index in boards.

In [ ]:
geom = CameraGeometry.from_name("DigiCam")
geom.rotate(90 * u.deg)
plt.figure(figsize=(12, 10))
disp = CameraDisplay(geom)
disp.image[pixels_ok] = offset_fit
disp.highlight_pixels(problematic_pixels, color='red', linewidth=3)
disp.set_limits_minmax(np.min(offset_fit), np.max(offset_fit))
disp.add_colorbar()
plt.title('measured time offset [ns]')
#disp.overlay_pixels_id()
pass

We check for parterns in the uncertancy of the measured offset.

In [ ]:
plt.figure(figsize=(12, 10))
disp = CameraDisplay(geom)
disp.image[pixels_ok] = d_offset_fit
disp.set_limits_minmax(np.min(d_offset_fit), np.max(d_offset_fit))
disp.highlight_pixels(problematic_pixels, color='red', linewidth=3)
cbar = disp.add_colorbar()
plt.title('uncertaincy on time offset [ns]')
pass

As we would like the uncertaincy for each pixel function of the charge, we will now fit the offset for all LEDs's AC levels.
Then we plot an histogram of the residuals of the parabola fit.

In [ ]:
points_rel = np.arange(-6, 7, 1)
n_point_fit = len(points_rel)
polys = np.ones([3, n_level, n_pixel]) * np.nan
residuals = np.ones([n_level, n_pixel]) * np.nan
index_offset_rel = np.ones([n_level, n_pixel]) * np.nan
for level_idx in range(n_level):
    bad_pixels = np.all(np.isnan(chi_squared[level_idx, :, :]), axis=-1)
    pixels_ok = np.arange(n_pixel)[~bad_pixels]
    print('level', AC_levels[level_idx], ':', len(pixels_ok), '/', n_pixel, 'fits are ok')
    chi_squared_level = chi_squared[level_idx, pixels_ok, :]
    index_best_offsets = np.nanargmin(chi_squared_level, axis=-1)
    index_best_offsets[index_best_offsets < -np.min(points_rel)] = -np.min(points_rel)
    index_best_offsets[index_best_offsets >= n_offset - np.max(points_rel)] =  n_offset - np.max(points_rel) -1
    fit_points_bool = np.zeros([len(pixels_ok), n_offset], dtype=bool)
    for k in points_rel:
        fit_points_bool = np.logical_or(fit_points_bool, np.eye(n_offset,k=k)[index_best_offsets])
    # we fit parabolas
    chi_squared_fit = chi_squared_level[fit_points_bool].reshape(len(pixels_ok), n_point_fit)
    polys_level, residuals_level, _, _, _ = np.polyfit(points_rel*delta_x, chi_squared_fit.transpose(), 2, full=True)
    polys[:, level_idx, pixels_ok] = polys_level
    residuals[level_idx, pixels_ok] = residuals_level
    index_offset_rel[level_idx, pixels_ok] = index_best_offsets
polys = polys.reshape(3, -1)
residuals = residuals.reshape(-1)

# we plot the residuals
plt.figure(figsize=(12, 6))
h, xh, _ = plt.hist(residuals[~np.isnan(residuals)], np.logspace(-2, 6, 200))
plt.xlabel('residuals')
plt.ylabel('counts')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')

# residuals function of min chi2
plt.figure(figsize=(12, 6))
plt.loglog(residuals, polys[2, :], '+')
pass

We calculate the offset and the uncertaincy as previously but for all light levels giving a reasonable fit.
Some aditional attention must be taken as we are fitting on less than optimal data.

There are for each pixel ~3 functional fits, and they are consistent with each other.

In [ ]:
#We discard the fits where the residuals are too large.
#fit_ok = np.logical_and(residuals < 3, np.min(chi_squared_fit, axis=1) < 10)
fit_ok = np.logical_and(residuals < 3, polys[2,:] < 20)
print(np.sum(fit_ok), '/', len(fit_ok.flatten()), 'fits succeded')
offset_fit_rel = np.ones([n_level* n_pixel]) * np.nan
offset_fit_rel[fit_ok] = (-0.5 * polys[1, fit_ok] / polys[0, fit_ok])
offset_fit = offset_fit_rel.reshape(n_level, n_pixel) + index_offset_rel * delta_x
d_offset_fit = np.ones([n_level* n_pixel]) * np.nan
d_offset_fit[fit_ok] = (np.sqrt(1./polys[0, fit_ok]))
d_offset_fit = d_offset_fit.reshape(n_level, n_pixel)
chi_squared_fit_min = polys[2, :].copy()
chi_squared_fit_min[~fit_ok] = np.nan
delta_chi_square = np.polyval(polys, offset_fit_rel - d_offset_fit.flatten()) - chi_squared_fit_min


# Now we check the hypothesis made to calculate the uncertaincy still holds. When too much off, we discard the fit.
fit_ok = np.logical_and(fit_ok, delta_chi_square > 0.8)
offset_fit_rel[~fit_ok] = np.nan
offset_fit.flatten()[~fit_ok] = np.nan
d_offset_fit.flatten()[~fit_ok] = np.nan
chi_squared_fit_min[~fit_ok] = np.nan
print(np.sum(fit_ok), '/', len(fit_ok), 'fits succeded after further checks')
std_offset_fitted = np.nanstd(offset_fit, axis=0)
std_offset_fitted_mean = np.nanmean(std_offset_fitted[std_offset_fitted>0])
print('the average over all pixels of the deviation of the fitted offset over the AC levels is:', std_offset_fitted)

plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(fit_ok))[fit_ok], delta_chi_square[fit_ok], '+')
plt.plot(np.arange(len(fit_ok))[~fit_ok], delta_chi_square[~fit_ok], '+')
plt.plot([0, len(fit_ok)], [1, 1], 'k--')
plt.title('$\Delta \chi^2$ at $1 \sigma$ (should be ~1)')
plt.xlabel('fit #')
plt.ylabel('$\Delta \chi^2$')

plt.figure(figsize=(12, 6))
plt.plot(offset_fit.transpose(), '+')
plt.xlabel('pixel #')
plt.ylabel('offset [ns]')

plt.figure(figsize=(12, 6))
plt.plot(d_offset_fit.transpose(), '+')
plt.plot(std_offset_fitted, '.k')
plt.xlabel('pixel #')
plt.ylabel('error on offset [ns]')
plt.yscale('log', nonposy='clip')

pass

Finally we plot :
- the uncertaincy function of the LEDs AC level
- the uncertaincy function of the charge

It is very uniform around 0.22 ns in the 20 - 700 p.e. range.

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(np.ones([n_pixel, 1]) * AC_levels, d_offset_fit, 5, np.log10(pe), '+')
plt.xlabel('LED AC level')
plt.ylabel('error on offset [ns]')
cbar = plt.colorbar()
cbar.set_label('$\log_{10}$(charge [p.e.])')

plt.figure(figsize=(12, 6))
plt.scatter(pe.flatten()[fit_ok], d_offset_fit.flatten()[fit_ok],5, np.log10(chi_squared_fit_min[fit_ok]), '+')
plt.xlim([1, 1000])
plt.ylim([0.1, 10])
cbar = plt.colorbar()
cbar.set_label('$\log_{10}(\chi^2)$')
plt.xscale('log', nonposx='clip')
plt.yscale('log', nonposy='clip')
plt.xlabel('measured charge [p.e.]')
plt.ylabel('error on offset [ns]')
pass